## UMAP redction of gametocyte data

In [1]:
import numpy as np
#from sklearn.datasets import load_iris, load_digits
#from sklearn.model_selection import train_test_split
import pandas as pd
import altair as alt
import umap

In [2]:
alt.data_transformers.disable_max_rows()
alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

### Load Data

In [3]:
data = pd.read_csv('../data/raw_data/rpkm_kent2018.tsv.gz', sep='\t')
data.head()

,gene_id,sample_id,logRPKM
0,PBANKA_0000101,SRX3653182_00h_yes_1,-2.686662
1,PBANKA_0000201,SRX3653182_00h_yes_1,-0.469591
2,PBANKA_0000301,SRX3653182_00h_yes_1,-1.135740
3,PBANKA_0000401,SRX3653182_00h_yes_1,-2.205069
4,PBANKA_0000600,SRX3653182_00h_yes_1,-2.004105


In [4]:
#data.set_index(['sample_id', 'gene'], inplace=True)

pivot_data = data.pivot(index='sample_id', columns='gene_id')['logRPKM']
pivot_data.head()

gene_id,PBANKA_0000101,PBANKA_0000201,PBANKA_0000301,PBANKA_0000401,PBANKA_0000600,PBANKA_0000701,PBANKA_0000801,PBANKA_0000901,PBANKA_0001001,PBANKA_0001101,...,PBANKA_MIT02800,PBANKA_MIT02900,PBANKA_MIT03000,PBANKA_MIT03100,PBANKA_MIT03200,PBANKA_MIT03300,PBANKA_MIT03400,PBANKA_MIT03500,PBANKA_MIT03600,PBANKA_MIT03700
sample_id,,,,,,,,,,,,,,,,,,,,,
SRX3653182_00h_yes_1,-2.686662,-0.469591,-1.135740,-2.205069,-2.004105,-2.141141,-2.079132,-2.524314,0.732392,-0.931419,...,3.707093,3.781823,2.467357,5.432145,6.788568,3.108479,2.808393,12.266816,4.830809,5.488142
SRX3653183_12h_no_1,-2.286016,-0.469591,-0.507701,-2.205069,0.532317,-2.141141,-2.079132,-2.123668,2.722345,0.834126,...,3.054034,3.071678,2.467357,3.698550,5.149188,3.736518,4.354655,7.815756,4.458334,5.370502
SRX3653184_24h_no_1,-2.157067,-0.469591,-0.970356,-2.205069,-0.046969,-2.141141,-2.079132,-2.524314,2.213271,-0.136195,...,3.394996,2.958822,2.756320,3.881795,3.869905,2.646695,3.884822,8.089233,3.453421,4.617670
SRX3653185_30h_no_1,-2.003229,-0.469591,-1.886487,-2.205069,0.616629,-2.141141,-2.079132,-2.524314,3.736866,0.926586,...,3.747766,2.815162,2.849170,3.743108,4.712845,3.688228,4.138889,8.920807,4.278629,4.685024
SRX3653186_12h_yes_1,-2.074363,-0.469591,-1.044079,-2.205069,0.228818,-2.141141,-2.079132,-2.524314,2.316712,-0.802693,...,2.996859,2.724802,2.467357,4.153449,5.398020,2.970031,4.673705,9.260443,3.726930,5.261372


In [7]:
samples = pd.read_csv('../data/raw_data/sample_sheet.tsv', sep='\t')
samples.sort_values(by=['stage'])
samples.head()

,sample_id,stage,rapa_treat,strand,fastq_r1,fastq_r2
0,SRX3653182_00h_yes_1,00h,yes,RF,sra/SRX3653182_1.fastq.gz,sra/SRX3653182_2.fastq.gz
1,SRX3653183_12h_no_1,12h,no,RF,sra/SRX3653183_1.fastq.gz,sra/SRX3653183_2.fastq.gz
2,SRX3653184_24h_no_1,24h,no,RF,sra/SRX3653184_1.fastq.gz,sra/SRX3653184_2.fastq.gz
3,SRX3653185_30h_no_1,30h,no,RF,sra/SRX3653185_1.fastq.gz,sra/SRX3653185_2.fastq.gz
4,SRX3653186_12h_yes_1,12h,yes,RF,sra/SRX3653186_1.fastq.gz,sra/SRX3653186_2.fastq.gz


### UMAP Test

In [8]:
reducer = umap.UMAP()

In [9]:
test_em = reducer.fit_transform(pivot_data)

/Users/rossmadden/anaconda3/envs/Kent2018_GametocyteDev/lib/python3.7/site-packages/umap/umap_.py:349: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "fuzzy_simplicial_set" failed type inference due to: Untyped global name 'nearest_neighbors': cannot determine Numba type of <class 'function'>

File "../../../anaconda3/envs/Kent2018_GametocyteDev/lib/python3.7/site-packages/umap/umap_.py", line 467:
def fuzzy_simplicial_set(
    <source elided>
    if knn_indices is None or knn_dists is None:
        knn_indices, knn_dists, _ = nearest_neighbors(
        ^

  @numba.jit()
/Users/rossmadden/anaconda3/envs/Kent2018_GametocyteDev/lib/python3.7/site-packages/numba/compiler.py:742: NumbaWarning: Function "fuzzy_simplicial_set" was compiled in object mode without forceobj=True.

File "../../../anaconda3/envs/Kent2018_GametocyteDev/lib/python3.7/site-packages/umap/umap_.py", line 350:
@numba.jit()
def fuzzy_simplicial_set(
^

  self.func_ir.

In [10]:
test_em[:, 0]
test_em[:, 1]

array([-13.422395 , -11.288554 , -12.728195 , -11.697249 , -10.931258 ,
        -9.147689 ,  -8.908545 , -13.2017   , -11.912776 , -12.351423 ,
       -11.519839 , -12.59364  , -11.15346  , -13.444162 , -10.375915 ,
       -10.61074  ,  -9.447631 ,  -9.710165 , -13.231276 , -10.697265 ,
       -12.607529 ,  -9.254604 , -12.092836 , -10.710796 , -11.0541315,
        -9.325481 , -12.243786 , -11.533511 ,  -9.996233 , -11.465281 ,
       -11.303307 , -11.919214 ], dtype=float32)

In [13]:
results = pd.DataFrame({
    # testing new near neighbour value.
#     'x': trans_test.embedding_[:, 0],
#     'y': trans_test.embedding_[:, 1], 
    'x': test_em[:, 0],
    'y': test_em[:, 1], 
    'sample_id': pivot_data.index
})

results = results.join(samples.set_index('sample_id'), on='sample_id')
results_all = results.join(pivot_data, on='sample_id')
results_all.head()

,x,y,sample_id,stage,rapa_treat,strand,fastq_r1,fastq_r2,PBANKA_0000101,PBANKA_0000201,...,PBANKA_MIT02800,PBANKA_MIT02900,PBANKA_MIT03000,PBANKA_MIT03100,PBANKA_MIT03200,PBANKA_MIT03300,PBANKA_MIT03400,PBANKA_MIT03500,PBANKA_MIT03600,PBANKA_MIT03700
0,-2.251430,-13.422395,SRX3653182_00h_yes_1,00h,yes,RF,sra/SRX3653182_1.fastq.gz,sra/SRX3653182_2.fastq.gz,-2.686662,-0.469591,...,3.707093,3.781823,2.467357,5.432145,6.788568,3.108479,2.808393,12.266816,4.830809,5.488142
1,-1.390402,-11.288554,SRX3653183_12h_no_1,12h,no,RF,sra/SRX3653183_1.fastq.gz,sra/SRX3653183_2.fastq.gz,-2.286016,-0.469591,...,3.054034,3.071678,2.467357,3.698550,5.149188,3.736518,4.354655,7.815756,4.458334,5.370502
2,-3.648969,-12.728195,SRX3653184_24h_no_1,24h,no,RF,sra/SRX3653184_1.fastq.gz,sra/SRX3653184_2.fastq.gz,-2.157067,-0.469591,...,3.394996,2.958822,2.756320,3.881795,3.869905,2.646695,3.884822,8.089233,3.453421,4.617670
3,-1.563758,-11.697249,SRX3653185_30h_no_1,30h,no,RF,sra/SRX3653185_1.fastq.gz,sra/SRX3653185_2.fastq.gz,-2.003229,-0.469591,...,3.747766,2.815162,2.849170,3.743108,4.712845,3.688228,4.138889,8.920807,4.278629,4.685024
4,-1.587931,-10.931258,SRX3653186_12h_yes_1,12h,yes,RF,sra/SRX3653186_1.fastq.gz,sra/SRX3653186_2.fastq.gz,-2.074363,-0.469591,...,2.996859,2.724802,2.467357,4.153449,5.398020,2.970031,4.673705,9.260443,3.726930,5.261372


In [14]:
melt_results = results_all.melt(id_vars=['sample_id', 'rapa_treat', 'x',  'y', 'stage'], value_vars=['strand'])
melt_results = melt_results.drop(['value', 'variable'], axis=1)
melt_results.head()

,sample_id,rapa_treat,x,y,stage
0,SRX3653182_00h_yes_1,yes,-2.251430,-13.422395,00h
1,SRX3653183_12h_no_1,no,-1.390402,-11.288554,12h
2,SRX3653184_24h_no_1,no,-3.648969,-12.728195,24h
3,SRX3653185_30h_no_1,no,-1.563758,-11.697249,30h
4,SRX3653186_12h_yes_1,yes,-1.587931,-10.931258,12h


In [15]:
final_data = data.set_index('sample_id').join(melt_results.set_index('sample_id'), on='sample_id')
final_data

,gene_id,logRPKM,rapa_treat,x,y,stage
sample_id,,,,,,
SRX3653182_00h_yes_1,PBANKA_0000101,-2.686662,yes,-2.251430,-13.422395,00h
SRX3653182_00h_yes_1,PBANKA_0000201,-0.469591,yes,-2.251430,-13.422395,00h
SRX3653182_00h_yes_1,PBANKA_0000301,-1.135740,yes,-2.251430,-13.422395,00h
SRX3653182_00h_yes_1,PBANKA_0000401,-2.205069,yes,-2.251430,-13.422395,00h
SRX3653182_00h_yes_1,PBANKA_0000600,-2.004105,yes,-2.251430,-13.422395,00h
SRX3653182_00h_yes_1,PBANKA_0000701,-2.141141,yes,-2.251430,-13.422395,00h
SRX3653182_00h_yes_1,PBANKA_0000801,-2.079132,yes,-2.251430,-13.422395,00h
SRX3653182_00h_yes_1,PBANKA_0000901,-2.524314,yes,-2.251430,-13.422395,00h
SRX3653182_00h_yes_1,PBANKA_0001001,0.732392,yes,-2.251430,-13.422395,00h


In [ ]:
# final_data.to_csv('../data/formatted_outputs/rpkm_data_umap_coords', sep='\t')

### Altair plots of UMAP redcution (not used further)

In [ ]:
scale=alt.Scale(domain=(6, 13)) 

no_rapa = alt.Chart(melt_results[melt_results['rapa_treat']=='no']).mark_point(size=60).encode( #[final_data['rapa_treat']=='no']
    alt.X('x', axis=alt.Axis(title=''), type='quantitative'),
    alt.Y('y', axis=alt.Axis(title=''), type='quantitative', scale=scale),
    tooltip = ['rapa_treat', 'stage'],
    color = 'stage:N'
)

rapa = alt.Chart(melt_results[melt_results['rapa_treat']=='yes']).mark_circle(size=60).encode(
    alt.X('x:Q'),
    alt.Y('y:Q', scale=scale),
    tooltip = ['rapa_treat', 'stage'],
    color = 'stage:N'
)

#input_dropdown = alt.binding_select(options=final_data['gene_id'].unique().tolist())
input_dropdown = alt.binding_select(options=['PBANKA_0000201', 'PBANKA_0000301'])
selection = alt.selection_single(fields=['gene_id'], bind=input_dropdown, name='Gene ID', clear='false')

# genre_dropdown = alt.binding_select(options=genres)
# genre_select = alt.selection_single(fields=['Major_Genre'], bind=genre_dropdown, name="Genre")


#both = alt.Chart('both_data.json').mark_circle(size=60).encode(
#both = alt.Chart(final_data[final_data['gene_id'].isin(['PBANKA_0000201', 'PBANKA_0000301'])]).mark_circle(size=60).encode(
both = alt.Chart(both_data).mark_circle(size=60).encode(   
    alt.X('x:Q', axis=alt.Axis(title='')), 
    alt.Y('y:Q', axis=alt.Axis(title=''), scale=scale),
    tooltip = ['logRPKM:Q',], # 'rapa_treat:O', 'stage:O'],
    color = alt.Color('logRPKM:Q', scale=alt.Scale(scheme='redblue')),
#    stroke = alt.value('black'),    
).add_selection(
    selection
).transform_filter(
    selection
)

both_circles = alt.Chart(melt_results).mark_point(size=65).encode( #[final_data['rapa_treat']=='no']
    alt.X('x', axis=alt.Axis(title=''), type='quantitative'),
    alt.Y('y', axis=alt.Axis(title=''), type='quantitative', scale=scale),
    color = alt.value('grey'),
    # tooltip = ['rapa_treat', 'stage'],
)

#selection = alt.selection_single(fields=['Gene'], bind=input_dropdown, name=' ', clear='false')

combined = no_rapa + rapa | both_circles + both
combined

#TODO load from json for both plot to get rpkm values. 

In [ ]:
input_dropdown = alt.binding_select(options=final_data['gene_id'].unique().tolist())
selection = alt.selection_single(fields=['gene_id'], bind=input_dropdown, name=' ', clear='false')

# genre_dropdown = alt.binding_select(options=genres)
# genre_select = alt.selection_single(fields=['Major_Genre'], bind=genre_dropdown, name="Genre")

# scale=alt.Scale(domain=(6, 12)),

both = alt.Chart(final_data).mark_circle(size=60).encode(
    alt.X('x', axis=alt.Axis(title='')), 
    alt.Y('y', axis=alt.Axis(title='')),
    tooltip = 'logRPKM:Q',
    color = 'logRPKM:Q',
     
).add_selection(
    selection
).transform_filter(
    selection
)


both.save('single_coloured.html')

In [ ]:
all_s = alt.Chart(melt_results).mark_point(size=60).encode( #[final_data['rapa_treat']=='no']
    alt.X('x', axis=alt.Axis(title=''), type='quantitative'),
    alt.Y('y', axis=alt.Axis(title=''), type='quantitative'),
    tooltip = 'sample_id:N',
    color = 'stage:N'
)

all_s 

### Bokeh plots of UMAP data

In [16]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.layouts import layout, column
from bokeh.models import ColumnDataSource, Div, Whisker
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.io import output_notebook, show, output_file
from bokeh.models import HoverTool, Legend

import pandas as pd
import numpy as np

In [17]:
output_notebook()

Loading BokehJS ...

In [19]:
# Adding colours to samples. 
colours = {'00h': '#005ab7', '06h': '#6a4c93', '12h': '#8ac926', '18h': '#ffca3a', '24h': '#fcab10', '30h': '#ff595e'}
melt_results['colours'] = melt_results['stage'].map(colours)
#melt_results.to_csv('../data/formatted_outputs/umap_data.tab', sep='\t')

In [20]:
melt_results.head()

,sample_id,rapa_treat,x,y,stage,colours
0,SRX3653182_00h_yes_1,yes,-2.251430,-13.422395,00h,#005ab7
1,SRX3653183_12h_no_1,no,-1.390402,-11.288554,12h,#8ac926
2,SRX3653184_24h_no_1,no,-3.648969,-12.728195,24h,#fcab10
3,SRX3653185_30h_no_1,no,-1.563758,-11.697249,30h,#ff595e
4,SRX3653186_12h_yes_1,yes,-1.587931,-10.931258,12h,#8ac926


In [ ]:
# Testing size
final_data.groupby(['x', 'y', 'rapa_treat']).size()

#### Plot of UMAP reduction - coloured by sample and logRPKM

In [21]:
source_xy = ColumnDataSource(final_data[(final_data['gene_id'] == 'PBANKA_0000301')])

source_xy_rapa = ColumnDataSource(melt_results[melt_results['rapa_treat'] == 'yes'])
source_xy_norapa = ColumnDataSource(melt_results[melt_results['rapa_treat'] == 'no'])

# Tools for the charts
hover_xy = HoverTool(
        tooltips=[
            ("Rapamycin Treatment", "@rapa_treat"),
            ("Stage", "@stage"),
        ]
    )

hover_rpkm = HoverTool(
        tooltips=[
            ("Rapamycin Treatment", "@rapa_treat"),
            ("Stage", "@stage"),
        ]
    )

base_plot_options = dict(width=400, plot_height=400, tools=["box_select, pan, reset, wheel_zoom"])

xy_plot_options = base_plot_options
xy_plot_options['tools'].append(hover_xy)

rpkm_plot_options = base_plot_options
rpkm_plot_options['tools'].append(hover_rpkm)


# Plotting the data - xy from umap
xy_data = figure(**xy_plot_options)
xy_data.circle('x', 'y', size=10, source=source_xy_rapa, line_width=0, color='colours')
xy_data.circle('x', 'y', size=10, source=source_xy_norapa, fill_alpha=0, line_width=2, color='colours')

# rpkm color mapper
max_c = max(source_xy.data['logRPKM'])
min_c = min(source_xy.data['logRPKM'])
print(max_c, min_c)

from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import linear_cmap 
from bokeh.palettes import RdBu
mapper = LinearColorMapper(palette='Viridis256', low=min_c, high=max_c)
#print(mapper)

# colormapper = linear_cmap(field_name = final_data[(final_data['gene_id'] == 'PBANKA_0000301')]['logRPKM'], 
#                           palette=RdBu, low=min_c, high=max_c)


# Plotting the data - rpkm
rpkm_data = figure(x_range=xy_data.x_range, y_range=xy_data.y_range ,**rpkm_plot_options, )
rpkm_data.circle('x', 'y', size=10, source=source_xy, line_color='grey', fill_color={'field': 'logRPKM', 'transform': mapper})

color_bar = ColorBar(color_mapper=mapper, width=8,  location=(0,0))
rpkm_data.add_layout(color_bar, 'right')

#geneid_input = TextInput(value="default", title="Label:")

# LinearColorMapper

# Putting plots together in a grid
p = gridplot([[xy_data, rpkm_data]])

show(p)

0.9137992499055121 -1.88648669370319


/Users/rossmadden/anaconda3/envs/Kent2018_GametocyteDev/lib/python3.7/site-packages/bokeh/plotting/helpers.py:628: UserWarning: HoverTool are being repeated
  warnings.warn("%s are being repeated" % ",".join(repeated_tools))


#### Plot of all genes 

##### UMAP Reduction

In [22]:
genes_umap = pivot_data.T
genes_umap.head()

sample_id,SRX3653182_00h_yes_1,SRX3653183_12h_no_1,SRX3653184_24h_no_1,SRX3653185_30h_no_1,SRX3653186_12h_yes_1,SRX3653187_24h_yes_1,SRX3653188_30h_yes_1,SRX3653189_00h_no_1,SRX3653190_24h_no_2,SRX3653191_00h_yes_2,...,SRX3653204_00h_yes_4,SRX3653205_06h_yes_2,SRX3653206_06h_yes_3,SRX3653207_24h_yes_3,SRX3653208_00h_no_3,SRX3653209_18h_no_2,SRX3653210_18h_yes_2,SRX3653211_06h_no_2,SRX3653212_06h_no_3,SRX3653213_24h_no_4
gene_id,,,,,,,,,,,,,,,,,,,,,
PBANKA_0000101,-2.686662,-2.286016,-2.157067,-2.003229,-2.074363,-2.686662,-2.686662,-2.208403,-2.686662,-2.686662,...,-2.686662,-1.341155,-1.061657,-2.686662,-0.977327,0.217513,0.014670,-0.930310,-0.983448,-1.722137
PBANKA_0000201,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,...,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591
PBANKA_0000301,-1.135740,-0.507701,-0.970356,-1.886487,-1.044079,-1.886487,0.272611,-1.408228,-0.846522,-1.886487,...,-1.886487,-1.886487,-1.281364,0.048106,-1.346439,-0.686623,-0.947383,0.227813,0.844836,-0.515931
PBANKA_0000401,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,...,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-1.643684,-2.205069,-2.205069
PBANKA_0000600,-2.004105,0.532317,-0.046969,0.616629,0.228818,-0.384343,-1.244292,-2.004105,1.314576,-0.582881,...,-0.216777,0.984195,1.371622,-1.447847,-0.107205,-0.445460,-0.096440,1.453935,2.409104,1.102138


In [ ]:
transform = umap.UMAP(n_neighbors=20, n_components=2, min_dist=0).fit(genes_umap) 

In [ ]:
genes_umap['x'] = transform.embedding_[:, 0]
genes_umap['y'] = transform.embedding_[:, 1]
#genes_umap.to_csv('../data/formatted_outputs/all_genes_umap' ,sep='\t')

In [25]:
genes_umap = pd.read_csv('../data/formatted_outputs/all_genes_umap', sep='\t')
genes_umap.head()

,gene_id,SRX3653182_00h_yes_1,SRX3653183_12h_no_1,SRX3653184_24h_no_1,SRX3653185_30h_no_1,SRX3653186_12h_yes_1,SRX3653187_24h_yes_1,SRX3653188_30h_yes_1,SRX3653189_00h_no_1,SRX3653190_24h_no_2,...,SRX3653206_06h_yes_3,SRX3653207_24h_yes_3,SRX3653208_00h_no_3,SRX3653209_18h_no_2,SRX3653210_18h_yes_2,SRX3653211_06h_no_2,SRX3653212_06h_no_3,SRX3653213_24h_no_4,x,y
0,PBANKA_0000101,-2.686662,-2.286016,-2.157067,-2.003229,-2.074363,-2.686662,-2.686662,-2.208403,-2.686662,...,-1.061657,-2.686662,-0.977327,0.217513,0.014670,-0.930310,-0.983448,-1.722137,-12.144781,0.040509
1,PBANKA_0000201,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,...,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-10.876478,0.609060
2,PBANKA_0000301,-1.135740,-0.507701,-0.970356,-1.886487,-1.044079,-1.886487,0.272611,-1.408228,-0.846522,...,-1.281364,0.048106,-1.346439,-0.686623,-0.947383,0.227813,0.844836,-0.515931,-10.958379,0.570054
3,PBANKA_0000401,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,...,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-1.643684,-2.205069,-2.205069,-12.968539,-0.323981
4,PBANKA_0000600,-2.004105,0.532317,-0.046969,0.616629,0.228818,-0.384343,-1.244292,-2.004105,1.314576,...,1.371622,-1.447847,-0.107205,-0.445460,-0.096440,1.453935,2.409104,1.102138,-10.371606,0.466263


In [ ]:
# For graph colour bar values. 
genes_umap.quantile(0.95)

In [26]:
data2 = pd.read_csv('../data/formatted_outputs/rpkm_data_umap_coords', sep='\t')
data2.head()
#data2.dtypes

,sample_id,gene_id,logRPKM,rapa_treat,x,y,stage
0,SRX3653182_00h_yes_1,PBANKA_0000101,-2.686662,yes,6.746709,-3.304976,00h
1,SRX3653182_00h_yes_1,PBANKA_0000201,-0.469591,yes,6.746709,-3.304976,00h
2,SRX3653182_00h_yes_1,PBANKA_0000301,-1.135740,yes,6.746709,-3.304976,00h
3,SRX3653182_00h_yes_1,PBANKA_0000401,-2.205069,yes,6.746709,-3.304976,00h
4,SRX3653182_00h_yes_1,PBANKA_0000600,-2.004105,yes,6.746709,-3.304976,00h


#### Reduction with scaled features

Previous reduction I did not scale the features (gene log RPKM vals) as they were on a similar scale. 

In [32]:
scaled_genes_umap = genes_umap.copy()
scaled_genes_umap = scaled_genes_umap.drop(['x', 'y'], axis=1)
scaled_genes_umap = scaled_genes_umap.set_index(['gene_id'])
scaled_genes_umap.head()

,SRX3653182_00h_yes_1,SRX3653183_12h_no_1,SRX3653184_24h_no_1,SRX3653185_30h_no_1,SRX3653186_12h_yes_1,SRX3653187_24h_yes_1,SRX3653188_30h_yes_1,SRX3653189_00h_no_1,SRX3653190_24h_no_2,SRX3653191_00h_yes_2,...,SRX3653204_00h_yes_4,SRX3653205_06h_yes_2,SRX3653206_06h_yes_3,SRX3653207_24h_yes_3,SRX3653208_00h_no_3,SRX3653209_18h_no_2,SRX3653210_18h_yes_2,SRX3653211_06h_no_2,SRX3653212_06h_no_3,SRX3653213_24h_no_4
gene_id,,,,,,,,,,,,,,,,,,,,,
PBANKA_0000101,-2.686662,-2.286016,-2.157067,-2.003229,-2.074363,-2.686662,-2.686662,-2.208403,-2.686662,-2.686662,...,-2.686662,-1.341155,-1.061657,-2.686662,-0.977327,0.217513,0.014670,-0.930310,-0.983448,-1.722137
PBANKA_0000201,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,...,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591,-0.469591
PBANKA_0000301,-1.135740,-0.507701,-0.970356,-1.886487,-1.044079,-1.886487,0.272611,-1.408228,-0.846522,-1.886487,...,-1.886487,-1.886487,-1.281364,0.048106,-1.346439,-0.686623,-0.947383,0.227813,0.844836,-0.515931
PBANKA_0000401,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,...,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-2.205069,-1.643684,-2.205069,-2.205069
PBANKA_0000600,-2.004105,0.532317,-0.046969,0.616629,0.228818,-0.384343,-1.244292,-2.004105,1.314576,-0.582881,...,-0.216777,0.984195,1.371622,-1.447847,-0.107205,-0.445460,-0.096440,1.453935,2.409104,1.102138


In [35]:
# Scaling
scaled_genes_umap = (scaled_genes_umap - scaled_genes_umap.mean()) / (scaled_genes_umap.max() - scaled_genes_umap.min())
scaled_genes_umap.head()

,SRX3653182_00h_yes_1,SRX3653183_12h_no_1,SRX3653184_24h_no_1,SRX3653185_30h_no_1,SRX3653186_12h_yes_1,SRX3653187_24h_yes_1,SRX3653188_30h_yes_1,SRX3653189_00h_no_1,SRX3653190_24h_no_2,SRX3653191_00h_yes_2,...,SRX3653204_00h_yes_4,SRX3653205_06h_yes_2,SRX3653206_06h_yes_3,SRX3653207_24h_yes_3,SRX3653208_00h_no_3,SRX3653209_18h_no_2,SRX3653210_18h_yes_2,SRX3653211_06h_no_2,SRX3653212_06h_no_3,SRX3653213_24h_no_4
gene_id,,,,,,,,,,,,,,,,,,,,,
PBANKA_0000101,-0.420262,-0.414692,-0.428507,-0.415538,-0.417925,-0.412484,-0.391380,-0.404548,-0.410497,-0.436344,...,-0.400723,-0.387097,-0.376103,-0.399168,-0.348472,-0.266765,-0.301619,-0.360179,-0.322034,-0.384735
PBANKA_0000201,-0.302279,-0.309384,-0.329753,-0.325422,-0.324363,-0.300957,-0.285648,-0.310627,-0.292673,-0.311931,...,-0.285699,-0.334921,-0.340439,-0.291487,-0.319316,-0.305152,-0.328150,-0.332853,-0.294367,-0.313642
PBANKA_0000301,-0.337728,-0.311594,-0.359058,-0.408678,-0.357857,-0.372232,-0.250253,-0.361327,-0.312705,-0.391441,...,-0.359209,-0.419744,-0.389337,-0.266343,-0.369667,-0.317277,-0.354326,-0.291490,-0.223595,-0.316273
PBANKA_0000401,-0.394634,-0.409999,-0.431316,-0.427398,-0.425545,-0.388258,-0.368413,-0.404368,-0.384903,-0.409319,...,-0.375737,-0.438816,-0.444977,-0.375777,-0.418971,-0.402109,-0.423228,-0.402490,-0.387810,-0.412145
PBANKA_0000600,-0.383939,-0.251298,-0.305020,-0.261597,-0.283644,-0.296669,-0.322593,-0.393513,-0.197856,-0.318289,...,-0.272582,-0.247890,-0.229532,-0.339000,-0.298508,-0.303803,-0.307707,-0.218767,-0.139371,-0.224434


In [36]:
scaled_transform = umap.UMAP(n_neighbors=20, n_components=2, min_dist=0).fit(scaled_genes_umap) 

/Users/rossmadden/anaconda3/envs/Kent2018_GametocyteDev/lib/python3.7/site-packages/umap/rp_tree.py:450: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "make_euclidean_tree" failed type inference due to: Cannot unify RandomProjectionTreeNode(array(int64, 1d, C), bool, none, none, none, none) and RandomProjectionTreeNode(none, bool, array(float32, 1d, C), float64, RandomProjectionTreeNode(array(int64, 1d, C), bool, none, none, none, none), RandomProjectionTreeNode(array(int64, 1d, C), bool, none, none, none, none)) for '$14.16', defined at /Users/rossmadden/anaconda3/envs/Kent2018_GametocyteDev/lib/python3.7/site-packages/umap/rp_tree.py (457)

File "../../../anaconda3/envs/Kent2018_GametocyteDev/lib/python3.7/site-packages/umap/rp_tree.py", line 457:
def make_euclidean_tree(data, indices, rng_state, leaf_size=30):
    <source elided>

        left_node = make_euclidean_tree(data, left_indices, rng_state, leaf_size)
        ^

[1] Dur

In [37]:
scaled_genes_umap['x'] = scaled_transform.embedding_[:,0]
scaled_genes_umap['y'] = scaled_transform.embedding_[:,1]

In [44]:
scaled_genes_umap.head()
# scaled_genes_umap.columns

,SRX3653182_00h_yes_1,SRX3653183_12h_no_1,SRX3653184_24h_no_1,SRX3653185_30h_no_1,SRX3653186_12h_yes_1,SRX3653187_24h_yes_1,SRX3653188_30h_yes_1,SRX3653189_00h_no_1,SRX3653190_24h_no_2,SRX3653191_00h_yes_2,...,SRX3653206_06h_yes_3,SRX3653207_24h_yes_3,SRX3653208_00h_no_3,SRX3653209_18h_no_2,SRX3653210_18h_yes_2,SRX3653211_06h_no_2,SRX3653212_06h_no_3,SRX3653213_24h_no_4,x,y
gene_id,,,,,,,,,,,,,,,,,,,,,
PBANKA_0000101,-0.420262,-0.414692,-0.428507,-0.415538,-0.417925,-0.412484,-0.391380,-0.404548,-0.410497,-0.436344,...,-0.376103,-0.399168,-0.348472,-0.266765,-0.301619,-0.360179,-0.322034,-0.384735,11.245164,-0.222015
PBANKA_0000201,-0.302279,-0.309384,-0.329753,-0.325422,-0.324363,-0.300957,-0.285648,-0.310627,-0.292673,-0.311931,...,-0.340439,-0.291487,-0.319316,-0.305152,-0.328150,-0.332853,-0.294367,-0.313642,10.065916,-0.691164
PBANKA_0000301,-0.337728,-0.311594,-0.359058,-0.408678,-0.357857,-0.372232,-0.250253,-0.361327,-0.312705,-0.391441,...,-0.389337,-0.266343,-0.369667,-0.317277,-0.354326,-0.291490,-0.223595,-0.316273,10.212925,-0.623337
PBANKA_0000401,-0.394634,-0.409999,-0.431316,-0.427398,-0.425545,-0.388258,-0.368413,-0.404368,-0.384903,-0.409319,...,-0.444977,-0.375777,-0.418971,-0.402109,-0.423228,-0.402490,-0.387810,-0.412145,12.289360,-0.298284
PBANKA_0000600,-0.383939,-0.251298,-0.305020,-0.261597,-0.283644,-0.296669,-0.322593,-0.393513,-0.197856,-0.318289,...,-0.229532,-0.339000,-0.298508,-0.303803,-0.307707,-0.218767,-0.139371,-0.224434,9.475740,-0.765969


In [50]:
scaled_data = ColumnDataSource(scaled_genes_umap.reset_index())


scaled_mapper = LinearColorMapper(palette='Magma256', low=-0.85, high=0.85)

scaled_genes_plot = figure(width=400, plot_height=400, tools=["box_select, pan, reset, wheel_zoom"])
scaled_genes_plot.circle('x', 'y', size=10, source=scaled_data, line_width=0, alpha=1, fill_color={'field': 'SRX3653199_30h_yes_2', 'transform': scaled_mapper})
show(scaled_genes_plot)

# Structure is the same as the data that is not scaled.


##### Data for sample logRPKM change over time (mean and std dev)

In [ ]:
mean_data1 = data2
mean_data1['stage'] = mean_data1['stage'].str.replace('h', '').astype('int32')

rapa_map = {'yes': 'Rapamycin', 'no': 'No Rapamycin'}

mean_data1['rapa_treat'] = mean_data1['rapa_treat'].map(rapa_map)

In [ ]:
mean_data2 = mean_data1.groupby(['gene_id', 'stage', 'rapa_treat']).mean()
mean_data3 = mean_data1.groupby(['gene_id', 'stage', 'rapa_treat']).std()
mean_data3['std'] = mean_data3['logRPKM']
mean_data3 = mean_data3.drop(['logRPKM', 'x', 'y'], axis=1)

In [ ]:
new = mean_data2.join(mean_data3, how='right', lsuffix='_mean', rsuffix='_std')
new['up_std'] = new['logRPKM'] + mean_data3['std']
new['low_std'] = new['logRPKM'] - mean_data3['std']
#new.to_csv('../data/formatted_outputs/rpkm_data_mean_stddev', sep='\t')
new

In [ ]:
new = pd.read_csv('../data/formatted_outputs/rpkm_data_mean_stddev', sep='\t')
new.head()

In [27]:
rapa_data = ColumnDataSource(mean_data2.loc['PBANKA_0000101', :, 'Rapamycin'].reset_index()) 
no_rapa_data = ColumnDataSource(mean_data2.loc['PBANKA_0000101', :, 'No Rapamycin'].reset_index())

p = figure(width=600, height=350, title='logRPKM Change Over Time')

p.xaxis.axis_label = 'Time (h)'
p.yaxis.axis_label = 'logRPKM'

rapa_l = p.line('stage', 'logRPKM', line_width=2, color='#ea2b1f', source=rapa_data, )
rapa_c = p.circle('stage', 'logRPKM', fill_color="white", size=8, color='#ea2b1f', source=rapa_data, )
no_rapa_l = p.line('stage', 'logRPKM', line_width=2, color='#5fa8d3', source=no_rapa_data, )
no_rapa_c = p.circle('stage', 'logRPKM', fill_color="white", size=8, color='#5fa8d3', source=no_rapa_data, )

legend = Legend(items=[
    ("Rapamycin"   , [rapa_l, rapa_c]),
    ("No Rapamycin" , [no_rapa_l, no_rapa_c]),


    ], location="center", orientation='horizontal')

p.add_layout(legend, 'below')


# w_r = Whisker(source=rapa_data, base='logRPKM', upper="std", lower="std", line_color='#ea2b1f',) #x_range_name='stage')
# w_r.upper_head.line_color = '#ea2b1f'
# w_r.lower_head.line_color = '#ea2b1f'

p.add_layout(w_r)
show(p)

NameError: name 'mean_data2' is not defined